In [4]:
import matplotlib.pyplot as plt
import numpy as np

In [5]:
class ValueMap:
    def __init__(self, gridSize, southWestPos, cellSize):
        self.gridSize = gridSize
        self.southWestPos = southWestPos
        self.cellSize = cellSize
        
        self.gridData = np.empty((gridSize[0], gridSize[1]))
    
    def setGridData(self, gridData):
        self.gridData = gridData
    
    def determineValue(self, lattitude, longitude):
        yIndex = self.gridData.shape[0] -  int(abs(lattitude - self.southWestPos[0]) / cellSize)
        xIndex = int(abs(longitude - self.southWestPos[1]) / cellSize)
        if yIndex > 0:
            yIndex = yIndex - 1
        if xIndex > 0:
            xIndex = xIndex - 1
        return self.gridData[yIndex][xIndex]
    
    def plot(self):
        plt.imshow(self.gridData, cmap='jet', interpolation='nearest')
        plt.colorbar()
        plt.show()

In [6]:
# Make maptlotlib show graphs in a new window
%matplotlib qt 

In [7]:
dataDate = "10/1/2024" # Date of the data to be collected in format %D/%M/%Y

gridData = []

In [8]:
with open(f"data/solarIradiance/{dataDate.replace('/', '')}.grid", "r", encoding="utf-8") as gridFile:
    gridData = gridFile.readlines()
    
for index, line in enumerate(gridData):
    gridData[index] = line.replace("\n", "")

In [9]:
gridSize = [-1,-1]
southWestPos = [-1,1]
cellSize = -1
noDataVal = -1
for line in gridData:
    if "ncols" in line:
        gridSize[1] = int(line[line.find(" ")+1:])
    if "nrows" in line:
        gridSize[0] = int(line[line.find(" ")+1:])
    if "cellsize" in line:
        cellSize = float(line[line.find(" ")+1:])
    if "nodata_value" in line:
        noDataVal = float(line[line.find(" ")+1:])
    if "SOUTHWEST LATITUDE" in line:
        line = line.replace(" ", "")
        southWestPos[0] = float(line[line.find("LATITUDE")+8:])
    if "SOUTHWEST LONGITUDE" in line:
        line = line.replace(" ", "")
        southWestPos[1] = float(line[line.find("LONGITUDE")+9:])

        

In [10]:
# Put grid data into a np array
mapData = np.empty((gridSize[0], gridSize[1]))

rowIndex = 0 #  
for line in gridData:
    row = line.split(" ")
    try:
        testVal = float(row[0]) # Ensure that the first value is a float, thus only considering values
        for itemIndex, item in enumerate(row):
            dataVal = float(item)
            if dataVal == noDataVal:
                mapData[rowIndex][itemIndex]=0
            else:
                mapData[rowIndex][itemIndex]=dataVal
        
        rowIndex = rowIndex + 1
    except ValueError:
        pass

irradianceVals = ValueMap(gridSize, southWestPos, cellSize)
irradianceVals.setGridData(mapData)

In [14]:
irradianceVals.determineValue(10.871634, 142.0799)

17.77

In [15]:
irradianceVals.plot()